# Load appids

In [8]:
import requests
import random
import numpy as np
import urllib.request
import pandas as pd
import json

response = requests.get('https://api.steampowered.com/ISteamApps/GetAppList/v2/')

apps = response.json()['applist']['apps']

# filter blank/non-english names
apps = [x for x in apps if x['name'] != '' and x['name'].isascii()]

rand_indices = random.sample(range(len(apps)-1), len(apps)-2)

# ---NEW---

In [9]:
import urllib.request
import pandas as pd
import numpy as np
import json

for i in range(0, 100000):
    df = pd.DataFrame(columns=["language", "review", "voted_up", "votes_up", "votes_funny", "weighted_vote_score", "comment_count", "steam_purchase", "received_for_free", "written_during_early_access"])
    app_id = apps[rand_indices[i]]['appid']
    app_name = apps[rand_indices[i]]['name']
    
    # ALL WITHIN FOR LOOP OF APPIDS
    with urllib.request.urlopen("http://store.steampowered.com/appreviews/"+str(app_id)+"?json=1") as url:
        d = json.load(url)
        url.close()


    total_reviews = d["query_summary"]["total_reviews"]
    total_adjust = total_reviews-total_reviews%100

    desired_iter = 100

    if total_adjust > desired_iter*100:
        max_iter = desired_iter
    else:
        max_iter = int(total_adjust/100)

    cursors = []
    cursorURL = ""
    cursor = ""

    
    if max_iter == 0:
        continue
    
    for j in range(0, max_iter):
        if cursor:
            cursorURL = "&cursor="+cursor

        with urllib.request.urlopen("http://store.steampowered.com/appreviews/"+str(app_id)+"?json=1&num_per_page=100"+cursorURL) as url:
            d = json.load(url)
            url.close()

            if "cursor" in d.keys():
                cursor = urllib.parse.quote(d["cursor"])
            else:
                break

            cursors.append(cursorURL)

    cursors = np.unique(cursors)

    for j in cursors:
        with urllib.request.urlopen("http://store.steampowered.com/appreviews/"+str(app_id)+"?json=1&num_per_page=100"+j) as url:
            d = json.load(url)
            url.close()

        for k in d["reviews"]:
            df.loc[len(df)] = [k["language"], k["review"], k["voted_up"], k["votes_up"], k["votes_funny"], k["weighted_vote_score"], k["comment_count"], k["steam_purchase"], k["received_for_free"], k["written_during_early_access"]]

    df = df.drop_duplicates()
    df.to_csv(f"data/{app_name}.csv", index=False)



KeyboardInterrupt: 